# 0.0 IMPORTS

## packages

In [7]:
import os, pickle, datetime, json
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models as ml
from keras import layers as l


pd.options.display.max_columns = 500 # this will set limit of columns to 500
pd.set_option('display.float_format', lambda x: '%.5f' % x)

%matplotlib inline
os.getcwd()

'p:\\Python\\GitHub\\airbnb_prediction'

## 0.1 Helper Function

## 0.2 Loading Data

In [8]:
df0 = pd.read_csv("data/train_users_2.csv")
df0.columns = df0.columns.str.lower()
df0.head(3)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.00000,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.00000,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US


In [9]:
df_sessions = pd.read_csv( "data/sessions.csv" )
df_sessions.columns = df_sessions.columns.str.lower()
df_sessions.head(3)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.00000
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.00000
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.00000


# 1.0 Data Description

In [10]:
df1 = df0.copy()

In [11]:
df1.shape

(213451, 16)

In [12]:
df1.isnull().sum()/len(df1)

id                        0.00000
date_account_created      0.00000
timestamp_first_active    0.00000
date_first_booking        0.58347
gender                    0.00000
age                       0.41223
signup_method             0.00000
signup_flow               0.00000
language                  0.00000
affiliate_channel         0.00000
affiliate_provider        0.00000
first_affiliate_tracked   0.02841
signup_app                0.00000
first_device_type         0.00000
first_browser             0.00000
country_destination       0.00000
dtype: float64

In [13]:
print( len(df1) )
df1 = df1.dropna()
print( len(df1) )

213451
68171


In [14]:
df1.dtypes

id                          object
date_account_created        object
timestamp_first_active       int64
date_first_booking          object
gender                      object
age                        float64
signup_method               object
signup_flow                  int64
language                    object
affiliate_channel           object
affiliate_provider          object
first_affiliate_tracked     object
signup_app                  object
first_device_type           object
first_browser               object
country_destination         object
dtype: object

In [15]:
for i in ["date_account_created", "date_first_booking"]:
    df1[i] = pd.to_datetime(df1[i])
df1[ 'timestamp_first_active' ] = pd.to_datetime( df1['timestamp_first_active'], format="%Y%m%d%H%M%S" )
df1['age'] = df1['age'].astype(int)
print( df1.dtypes )

id                                 object
date_account_created       datetime64[ns]
timestamp_first_active     datetime64[ns]
date_first_booking         datetime64[ns]
gender                             object
age                                 int32
signup_method                      object
signup_flow                         int64
language                           object
affiliate_channel                  object
affiliate_provider                 object
first_affiliate_tracked            object
signup_app                         object
first_device_type                  object
first_browser                      object
country_destination                object
dtype: object


# 2.0 Data Filtering

In [16]:
df2 = df1.copy()

# 3.0 Data Prep

In [23]:
df3 = df2.copy()

#dummy variable
df3_dummy = pd.get_dummies( df3.drop( ['id', 'country_destination'], axis=1 ) )

#join id and country destination
df3 = pd.concat( [df3[['id', 'country_destination']], df3_dummy], axis=1 )

# 4.0 Feature Selection

In [24]:
cols_drop = ['date_account_created', 'timestamp_first_active', "date_first_booking"]
df4 = df3.drop( cols_drop , axis=1)

# 5.0 Neural Network - Multi-Layer Perceptron

In [26]:
df5 = df4.copy()

In [27]:
X = df5.drop( ["country_destination","id"] , axis=1 )
y = df5['country_destination'].copy()

In [28]:
Xtrain, Xtest, ytrain, ytest = train_test_split( X, y, test_size = 0.2, random_state=42 )

In [29]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ytrain_nn = ohe.fit_transform( ytrain.values.reshape( -1, 1 ) ).toarray()


In [31]:
model = ml.Sequential()
model.add( l.Dense( 128, input_dim = Xtrain.shape[1], activation = "relu" ) )
model.add( l.Dense( 11, activation="softmax") )

#model compile
model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

#train model
model.fit( Xtrain, ytrain_nn, epochs=100 )

Epoch 1/100
1705/1705 [==============================] - 4s 2ms/step - loss: 1.3940 - accuracy: 0.7013
Epoch 2/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.2850 - accuracy: 0.7062
Epoch 3/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.2591 - accuracy: 0.7068
Epoch 4/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.2184 - accuracy: 0.7071
Epoch 5/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.1986 - accuracy: 0.7070
Epoch 6/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.1808 - accuracy: 0.7071
Epoch 7/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.1636 - accuracy: 0.7071
Epoch 8/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.1544 - accuracy: 0.7071
Epoch 9/100
1705/1705 [==============================] - 3s 2ms/step - loss: 1.1462 - accuracy: 0.7072
Epoch 10/100
1705/1705 [==============================] - 3s 2ms/step - l

# 6.0 NN Performance

In [35]:
# prediction
pred_nn = model.predict( Xtest )

# invert prediction
yhat_nn = ohe.inverse_transform( pred_nn )

# prediction prepare
ytest_nn = ytest.to_numpy()
yhat_nn = yhat_nn.reshape( 1, -1 )[0]

427/427 [==============================] - 1s 1ms/step


In [42]:
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, accuracy_score, classification_report

In [41]:
# accuracy

acc_nn = accuracy_score(ytest_nn, yhat_nn)

bacc_nn = balanced_accuracy_score(ytest_nn, yhat_nn)

kappa_nn = cohen_kappa_score(ytest_nn, yhat_nn)

print( acc_nn, "\n", bacc_nn, "\n", kappa_nn)

0.7068573524019068 
 0.09175116530287762 
 0.0055010028822155554


In [44]:
print(classification_report(ytest_nn, yhat_nn))

p:\Python\GitHub\airbnb_prediction\airbnb_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AU       0.00      0.00      0.00        90
          CA       0.00      0.00      0.00       210
          DE       0.00      0.00      0.00       176
          ES       0.11      0.00      0.01       344
          FR       0.12      0.00      0.00       742
          GB       0.00      0.00      0.00       356
          IT       0.00      0.00      0.00       399
          NL       0.00      0.00      0.00       107
          PT       0.00      0.00      0.00        26
          US       0.71      0.99      0.83      9704
       other       0.18      0.01      0.03      1481

    accuracy                           0.71     13635
   macro avg       0.10      0.09      0.08     13635
weighted avg       0.54      0.71      0.59     13635



p:\Python\GitHub\airbnb_prediction\airbnb_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
p:\Python\GitHub\airbnb_prediction\airbnb_env\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
